#### 大問2. 乳がんデータセット
================

## Q1. 実装
つぎの条件にしたがって、乳がん(breast_cancer)の分類を行ってください。

なお、指定された条件以外にも、必要だと思われる処理等を思いついた場合に自由に追加してもらって構いません。

### 条件
- 使用するデータ: [breast_cancer dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html)
- 評価指標: 正答率(Accuracy)
- 評価プロトコル: ホールドアウト(交差検証は不要)
- アルゴリズム: サポートベクターマシン(Support Vector Machine)

In [44]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

cancer_data = load_breast_cancer()

#データの確認
print(cancer_data.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [0]:
X = pd.DataFrame(cancer_data.data, columns=cancer_data.feature_names)

y = pd.Series(cancer_data.target)

In [46]:
y.value_counts()

1    357
0    212
dtype: int64

In [47]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
mean radius                569 non-null float64
mean texture               569 non-null float64
mean perimeter             569 non-null float64
mean area                  569 non-null float64
mean smoothness            569 non-null float64
mean compactness           569 non-null float64
mean concavity             569 non-null float64
mean concave points        569 non-null float64
mean symmetry              569 non-null float64
mean fractal dimension     569 non-null float64
radius error               569 non-null float64
texture error              569 non-null float64
perimeter error            569 non-null float64
area error                 569 non-null float64
smoothness error           569 non-null float64
compactness error          569 non-null float64
concavity error            569 non-null float64
concave points error       569 non-null float64
symmetry error             569 

In [48]:
X.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [49]:
#データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

#スケール
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)

#SVM
svc = SVC()
svc.fit(X_train_sc, y_train)

#スケール
X_test_sc = scaler.transform(X_test)

#評価
print(f"{svc.score(X_test_sc, y_test):.2f}")



0.95


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


## Q2. 評価その1
クライアントに説明することを想定して、最終的な評価や分析結果も記述してください。


- 手持ちのデータ５６９件からSVMをモデルとして、９５％の正答率が得られた
- データの欠損はないが、値の大きさに違いがあるためスケールを調整
- 本来、乳がんの判定に正答率でモデルを評価するのは好ましくない

## Q3. モデルの改良
クライアントから「正答率は最低でも90%以上は欲しい」という要望がありました。

正答率が90%以上になるようにモデルを改良してください。このとき、Q1の条件は自由に変更してもらって構いません。

なお、Q1の時点で既に90%以上の正答率を出している場合には、Q3およびQ4の回答は不要です。


## Q4. 評価その2
あらためて、クライアントに説明することを想定して、最終的な評価や分析結果も記述してください。

## Q5. 混同行列による評価
今回のモデルにおける混同行列を求め、その結果についての説明をしてください。


In [0]:
from sklearn.metrics import confusion_matrix

svc_pred = svc.predict(X_test)
confusion = confusion_matrix(y_test, svc_pred)

In [0]:
# 悩みだしたのでいったん止め